In [ ]:
#from src.models.unet import UNet
from src.train import Train
from src.general import iterate_dict, create_checkpoints_folders
from src.dataset_pytorch import DatasetRetriever
from src.model_generator import ModelGenerator

import os
import torch
import warnings

In [ ]:
warnings.filterwarnings("ignore", message="Applied workaround for CuDNN issue") #ignora o warn do cudnn

In [ ]:
BS   = 32
WKRS = 2

INFOS        = {
                "checkpoints_folder": "checkpoints",
                "dataset_name": "mirc",
                "subset": "multi",
                "task": "segmentation",
                "learning": "dl"
                }

IMG_DIR      = os.path.join("dataset", INFOS["dataset_name"], "selected_frames", INFOS["subset"]) + os.sep
MASK_DIR     = os.path.join("dataset", INFOS["dataset_name"], "annotations", INFOS["subset"]) + os.sep
VAL_SIZE     = 0.2 #valor de produção é 0.2 e lembrar de colocar rapid tests = False
NUM_CHANNELS = 3
NUM_CLASSES  = 2
MODELS_LIST  = ["unet"]
EPOCHS       = 5

In [ ]:
#cria as pastas do chekcpoint
create_checkpoints_folders(INFOS, MODELS_LIST)

In [ ]:
#device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Criação dos modelos
mg = ModelGenerator(MODELS_LIST, n_channels=NUM_CHANNELS, n_classes=NUM_CLASSES)

# Criar o DataLoader
dr = DatasetRetriever(IMG_DIR, MASK_DIR, BS, VAL_SIZE, rapid_tests=False)
dataset_info = dr.get_dataloaders()

In [ ]:
#módulo de treinamento
for model in iterate_dict(mg.models):
    train = Train(model, device, EPOCHS, BS, dataset_info, INFOS)
    train.fit()